In [9]:
import json
import os
import sys
import pickle
import re

import numpy as np
import pandas as pd
from networks import DualNet, DualNetEndToEnd, PrimalNet, PrimalNetEndToEnd
import torch


data_path = f"experiment-output/ch6/ED_NB-G-F_GB2-G2-F2_L3_c0_s0_p0_smp15.pkl"
data = pickle.load(open(data_path, 'rb'))

indices = torch.arange(data.X.shape[0])

repeats = 5

stats_dict = {}

for run, (path, name) in enumerate(zip([
            # "experiment-output/ch5/plain-PDL/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746265816-296861",
            # "experiment-output/ch5/repair-1/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746429988-424426",
            "experiment-output/ch5/repair-2/learn_primal:True_train:0.8_rho:0.5_rhomax:5000_alpha:10_L:10-1746434902-974866"],
            [
                "plain",
                "repair1",
                "repair2"
                ])):
    stats_dict[name] = {"diff_p_gt": [], "diff_f_lt": [], "diff_md_nt": [], "diff_net_flow": [], "cheapest_generator_fraction": []}
    with open(os.path.join(path, 'args.json'), 'r') as f:
        args = json.load(f)
    # Compute sizes for each set
    train_size = int(args["train"] * data.X.shape[0])
    valid_size = int(args["valid"] * data.X.shape[0])
    # print(f"Train size: {train_size}, Valid size: {valid_size}, Test size: {data.X.shape[0] - train_size - valid_size}")


    # Split the indices
    train_indices = indices[:train_size]
    valid_indices = indices[train_size:train_size+valid_size]
    test_indices = indices[train_size+valid_size:]

    for i, repeat in enumerate(range(repeats)):
        
        directory = os.path.join(path, f"repeat:{repeat}")
        # directory = f"experiment-output/ch5-reproduction-nonconvex/{experiment}/repeat:{repeat}"
        # dual_net = DualNet(args, data=data)
        # dual_net.load_state_dict(torch.load(os.path.join(directory, 'dual_weights.pth'), weights_only=True))

        primal_net = PrimalNetEndToEnd(args, data=data)
        primal_net.load_state_dict(torch.load(os.path.join(directory, 'primal_weights.pth'), weights_only=True))
        X_test = data.X[test_indices]
        Y = primal_net(X_test)
        p_gt, f_lt, md_nt = data.split_dec_vars_from_Y(Y)
        p_gt_target, f_lt_target, md_nt_target = data.split_dec_vars_from_Y(data.opt_targets["y_operational"][test_indices])
        diff_p_gt = p_gt - p_gt_target
        diff_f_lt = f_lt - f_lt_target
        diff_md_nt = md_nt - md_nt_target

        net_flow = data.net_flow(f_lt)
        net_flow_target = data.net_flow(f_lt_target)
        diff_net_flow = net_flow - net_flow_target

        costs = data.cost_vec
        generators = ["BEL-WindOff", "BEL-Gas", "GER-Gas", "GER-SunPV", "FRA-Nuclear", "FRA-SunPV"]
        cheapest_generators_indices = [0, 3, 5] # BEL-WindOff, GER-SunPV, FRA-SunPV
        eq_rhs, ineq_rhs = data.split_X(X_test)
        p_gt_ub = ineq_rhs[:, data.capacity_ub_indices]
        demand = eq_rhs

        effective_demand = demand - net_flow

        cap = torch.min(p_gt_ub[:, cheapest_generators_indices], effective_demand)
        diff = (cap - p_gt[:, cheapest_generators_indices])
        fraction = diff / torch.where(cap == 0, torch.ones_like(cap), cap)
        print(fraction.mean().item())

        
        # cheapest_generator_fraction = data.cost_vec

        stats_dict[name]["diff_p_gt"].append(diff_p_gt.abs().mean().item())
        stats_dict[name]["diff_f_lt"].append(diff_f_lt.abs().mean().item())
        stats_dict[name]["diff_md_nt"].append(diff_md_nt.abs().mean().item())
        stats_dict[name]["diff_net_flow"].append(diff_net_flow.abs().mean().item())
        stats_dict[name]["cheapest_generator_fraction"].append(fraction.mean().item())


stats_dict


-49.90105061012176
-58.91226511042528
-47.921306094546
-45.96869649013062
-50.64556047482254
0.40812335738910627
0.44547749360923233
0.40370662507099686
0.43780031072718784
0.46232394769137786
8.256309104656675e-13
2.7156688389057762e-11
4.394320472737566e-07
2.250474916434122e-12
3.041330670556621e-08


{'plain': {'diff_p_gt': [10075.475045903975,
   10004.315618317654,
   10179.293396331419,
   10047.53476991365,
   10098.479836578315],
  'diff_f_lt': [1343.156907958611,
   1356.1407105858486,
   1367.4201764725065,
   1401.2697301379567,
   1391.5394267149773],
  'diff_md_nt': [18490.90492003821,
   18500.852954332062,
   18716.054191817708,
   18458.031978107476,
   18592.37045768122],
  'diff_net_flow': [2136.0289176787105,
   2161.3123978706903,
   2164.63218336952,
   2234.156170104473,
   2261.4196811809525],
  'cheapest_generator_fraction': [-49.90105061012176,
   -58.91226511042528,
   -47.921306094546,
   -45.96869649013062,
   -50.64556047482254]},
 'repair1': {'diff_p_gt': [3974.1528328290456,
   4181.689459800387,
   3989.098713051427,
   4150.2755811507295,
   4240.989190936368],
  'diff_f_lt': [3286.6591298079716,
   3122.5127037874977,
   2861.358196424854,
   3023.369516608,
   3178.379504149183],
  'diff_md_nt': [1620.7183056929,
   1787.41541250516,
   1527.86174143

In [10]:
# Prepare separate dictionaries
summary = {"Experiment": [], "Production diff.": [], "Unmet demand diff.": [], "Net flow diff.": [], "Expensive generation (\%)": []}

# {"diff_p_gt": [], "diff_f_lt": [], "diff_md_nt": [], "diff_net_flow": [], "cheapest_generator_fraction": []}

for experiment, metrics in stats_dict.items():
    summary["Experiment"].append(experiment)
    summary["Production diff."].append(f"{np.mean(metrics['diff_p_gt']):.3f}({np.std(metrics['diff_p_gt']):.3f})")
    summary["Unmet demand diff."].append(f"{np.mean(metrics['diff_md_nt']):.3f}({np.std(metrics['diff_md_nt']):.3f})")
    summary["Net flow diff."].append(f"{np.mean(metrics['diff_net_flow']):.3f}({np.std(metrics['diff_net_flow']):.3f})")
    summary["Expensive generation (\%)"].append(f"{np.mean(metrics['cheapest_generator_fraction']):.3f}({np.std(metrics['cheapest_generator_fraction']):.3f})")
    # summary["Optimal Obj"].append(f"{np.mean(metrics['known_obj']):.3f}")
    # summary["Predicted Obj"].append(f"{np.mean(metrics['predicted_obj']):.3f}({np.std(metrics['predicted_obj']):.3f})")
    # summary["OptGap (%)"].append(f"{np.mean(metrics['opt_gap']):.3f}({np.std(metrics['opt_gap']):.3f})")

# Convert to DataFrames
df = pd.DataFrame(summary)

# Export LaTeX tables
latex = df.to_latex(index=False, escape=False)

# Print or write to file
print(latex)

\begin{tabular}{lllll}
\toprule
Experiment & Production diff. & Unmet demand diff. & Net flow diff. & Expensive generation (\%) \\
\midrule
plain & 10081.020(58.306) & 18551.643(93.511) & 2191.510(47.788) & -50.670(4.431) \\
repair1 & 4107.241(106.727) & 1722.825(131.056) & 4700.549(53.357) & 0.431(0.022) \\
repair2 & 133.665(41.370) & 20.533(0.694) & 287.862(82.972) & 0.000(0.000) \\
\bottomrule
\end{tabular}

